In [1]:
import cv2
import pika
import numpy as np
import json
import base64
import time

In [2]:

video_capture = cv2.VideoCapture(-1)

while not video_capture.isOpened():
    time.sleep(1)
    video_capture.open()

In [3]:
parameters = pika.URLParameters('amqp://localhost:5672')
connection = pika.BlockingConnection(parameters=parameters)
channel = connection.channel()
ex = 'camera_capture'
channel.exchange_declare(exchange=ex, exchange_type='topic')

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>

In [4]:
reading = True
while(reading):
    ret, frame = video_capture.read()
    if ret:
        frame = (np.array(frame)/255.0).astype(dtype=np.float32)
        frame_bytes = frame.tobytes()
        frame_b6 = base64.encodebytes(frame_bytes)
        json_formatted = frame_b6.decode('utf-8')
        channel.basic_publish(exchange=ex, routing_key='capture', body=json.dumps({'image':json_formatted,'height':frame.shape[0], 'width':frame.shape[1], 'channels':frame.shape[2]}))
    else:
        print("Could not read from capture")
    time.sleep(.5)

KeyboardInterrupt: 